In [2]:
import torch
from torchmetrics.regression import ConcordanceCorrCoef, MeanSquaredError

In [3]:
outputs = torch.rand(4, 10, 20, 30, 3)
labels = torch.rand(4, 10, 20, 30, 3)

In [4]:
opts = torch.flatten(outputs)
labs = torch.flatten(labels)

In [5]:
opts.shape

torch.Size([72000])

In [6]:
mse = MeanSquaredError()

In [7]:
mse(opts, labs)

tensor(0.1662)

In [8]:
torch.mean(mse(opts, labs))

tensor(0.1662)

In [9]:
opts_b = torch.flatten(outputs, start_dim=1)
labs_b = torch.flatten(labels, start_dim=1)

In [10]:
opts_b.shape

torch.Size([4, 18000])

In [11]:
loss_mse = 0.0
for i in range(opts_b.shape[0]):
    loss_mse += torch.mean(mse(opts_b[i], labs_b[i]))
loss_mse /= opts_b.shape[0]
loss_mse

tensor(0.1662)

In [12]:
ccc = ConcordanceCorrCoef()

In [13]:
torch.mean(torch.square(1.0 - ccc(opts, labs)))

tensor(0.9938)

In [14]:
b = torch.permute(outputs, (0, 3, 2, 1, 4))
B = torch.permute(labels, (0, 3, 2, 1, 4))

In [15]:
b.shape

torch.Size([4, 30, 20, 10, 3])

In [16]:
b[:, :, :, 0, :].shape

torch.Size([4, 30, 20, 3])

In [17]:
B[:, :, :, 0, :].shape

torch.Size([4, 30, 20, 3])

In [18]:
torch.mean(torch.square((b[:, :, :, 0, :] - B[:, :, :, 0, :])))

tensor(0.1707)

In [19]:
mse(b[:, :, :, 0, :].flatten(), B[:, :, :, 0, :].flatten())

tensor(0.1707)

In [20]:
from rtmag.train.diff_torch_batch import curl, divergence

In [21]:
d = torch.FloatTensor([1.0])

In [22]:
bx, by, bz = b[..., 0], b[..., 1], b[..., 2]
jx, jy, jz = curl(bx, by, bz, d, d, d)
bb = torch.stack([bx, by, bz], -1)
j = torch.stack([jx, jy, jz], -1)

In [23]:
torch.allclose(b, bb)

True

In [24]:
jxb = torch.cross(j, bb, -1)
jxb.shape

torch.Size([4, 30, 20, 10, 3])

In [25]:
loss_ff = (jxb**2).sum(-1) / ((b**2).sum(-1) + 1e-7)
loss_ff.shape

torch.Size([4, 30, 20, 10])

In [26]:
torch.mean(loss_ff)

tensor(0.4110)

In [27]:
div_b = divergence(bx, by, bz, d, d, d)
div_b.shape

torch.Size([4, 30, 20, 10])

In [28]:
loss_div = torch.mean(torch.square(div_b))

In [29]:
loss_div

tensor(0.3086)

In [30]:
mse(div_b.flatten(), torch.zeros_like(div_b).flatten())

tensor(0.3086)

In [31]:
Bx, By, Bz = B[..., 0], B[..., 1], B[..., 2]
Jx, Jy, Jz = curl(Bx, By, Bz, d, d, d)
J = torch.stack([Jx, Jy, Jz], -1)

In [32]:
J.shape

torch.Size([4, 30, 20, 10, 3])

In [33]:
(b[0][0][0][0]**2).sum()

tensor(1.6560)

In [34]:
(b**2).sum(-1)[0][0][0][0]

tensor(1.6560)

In [35]:
b_energy = torch.sum(b**2, -1)
B_energy = torch.sum(B**2, -1)

In [36]:
torch.mean(torch.square(b_energy - B_energy))

tensor(0.5292)

In [37]:
torch.mean(torch.square(J - j))

tensor(0.4083)

In [38]:
(b**2).sum(-1).sum()

tensor(23980.6328)

In [39]:
(b**2).sum(-1).sum() / (B**2).sum(-1).sum()

tensor(0.9977)

In [44]:
torch.mean(torch.sum(jxb**2, -1))

tensor(0.4225)

In [45]:
torch.mean(torch.square(torch.sum(jxb**2, -1)))

tensor(0.8952)

In [46]:
torch.abs(1.0 - ccc(opts, labs))

tensor(0.9969)

In [47]:
torch.square(1.0 - ccc(opts, labs))

tensor(0.9938)

In [48]:
(1.0 - 0.95)

0.050000000000000044

In [49]:
(1.0 - 0.95)**2

0.0025000000000000044

In [50]:
torch.mean(torch.square(opts - labs))

tensor(0.1662)

In [51]:
torch.mean(torch.abs(opts - labs))

tensor(0.3327)

In [60]:
N = 10 * 20 * 30 * 3

In [61]:
torch.sum(torch.square(opts - labs)) / N

tensor(0.6647)

In [62]:
torch.sum(torch.square(opts - labs)) / (4*N)

tensor(0.1662)

In [52]:
a = torch.Tensor([1, 1, 10, 1, 1])
b = torch.Tensor([1, 1, 1, 1, 1])

In [58]:
torch.mean(torch.square(a - b))

tensor(16.2000)

In [54]:
torch.mean(torch.abs(a - b))

tensor(1.8000)

In [59]:
torch.square(a-b)

tensor([ 0.,  0., 81.,  0.,  0.])

tensor(0.0045)